## Setting up

In [1]:
%pip install tensorflow==2.16.1 nibabel matplotlib numpy opencv-python image-classifiers==1.0.0b1 keras_applications keras_preprocessing keras_cv


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from glob import glob 
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2

In [3]:
input_shape = (512, 512, 1)

## Preprocessing

In [4]:
def read_nifti_file(filepath):
  """Read and load volume"""
  # Read file
  scan = nib.load(filepath)
  # Get raw data
  scan = scan.get_fdata()
  return scan


def load_data(path):
  images = sorted(glob(os.path.join(path, 'rp_im', '*')))
  lung_masks = sorted(glob(os.path.join(path, 'rp_lung_msk', '*')))
  masks = sorted(glob(os.path.join(path, 'rp_msk', '*')))

  return (images, lung_masks, masks)


def parse(images, lung_masks, masks):
  images = read_nifti_file(images)
  lung_masks = read_nifti_file(lung_masks)
  masks = read_nifti_file(masks)
  return images, lung_masks, masks

def to_uint8(data):
    data -= data.min()
    data /= data.max()
    data *= 255
    return data.astype(np.uint8)

In [5]:
%rm -r 'dataset/converted/'

%mkdir 'dataset/converted/'
%mkdir 'dataset/converted/images'
%mkdir 'dataset/converted/masks'

In [6]:
images, lung_masks, masks = load_data('dataset/MedSeg Covid Dataset 2')

for image, lung_mask, mask in zip(images, lung_masks, masks):
  image_slices, lung_mask_slices, mask_slices = parse(image, lung_mask, mask)
  for i in range(image_slices.shape[2]):
    image_slice = to_uint8(image_slices[:, :, i])
    image_slice = np.rot90(image_slice, k=1, axes=(1, 0))

    lung_mask_slice = np.uint8(lung_mask_slices[:, :, i])
    lung_mask_slice = np.rot90(lung_mask_slice, k=1, axes=(1, 0))

    mask_slice = np.uint8(mask_slices[:, :, i])
    mask_slice = np.rot90(mask_slice, k=1, axes=(1, 0))

    p = np.where(mask_slice != 0)
    p1 = np.where(lung_mask_slice == 1)
    p2 = np.where(mask_slice > 0)

    mask_slice[p] = 0
    mask_slice[p1] = 1
    mask_slice[p2] = 2

    if (0 in np.unique(mask_slice)):
      cv2.imwrite(f'dataset/converted/images/{os.path.splitext(os.path.basename(image))[0]}_{i}.png', image_slice)
      cv2.imwrite(f'dataset/converted/masks/{os.path.splitext(os.path.basename(image))[0]}_{i}.png', mask_slice)

In [ ]:
%rm -r 'dataset/test'
%rm -r 'dataset/train'
%rm -r 'dataset/val'

%mkdir 'dataset/test'
%mkdir 'dataset/test/images'
%mkdir 'dataset/test/masks'

%mkdir 'dataset/train'
%mkdir 'dataset/train/images'
%mkdir 'dataset/train/masks'

%mkdir 'dataset/val'
%mkdir 'dataset/val/images'
%mkdir 'dataset/val/masks'